In [14]:
# pip install hid
import hid

vendor_id = 0x17EF
product_id_match = lambda x: x & 0xFFF0 == 0x6180
usage_page = 0xFFA0

config = None
for dev in hid.enumerate(vendor_id):
    if product_id_match(dev["product_id"]) and dev["usage_page"] == usage_page:
        config = dev
        break

if not config:
    print("Legion go configuration device not found.")
else:
    print(config)

{'path': b'/dev/hidraw6', 'vendor_id': 6127, 'product_id': 24962, 'serial_number': '32869681', 'release_number': 256, 'manufacturer_string': ' ', 'product_string': 'Legion Controller for Windows ', 'usage_page': 65440, 'usage': 1, 'interface_number': 2}


In [12]:

# Command to disable the touchpad
# Format: 0506 6b 0204 <enable> 01
command = [
    0x05,
    0x06,  # Report ID and Length
    0x6B,  # Command (Nibble 6 + b)
    0x02,  # Command sub-parameter
    0x04,  # Right Controller
    0x01,  # Disable flag (0x00 for disable, 0x01 for enable)
    0x01,  # All commands end with 0x01
]

byte_command = bytes(command)

# Pad the byte_command with 0xCD to meet the length of 64 bytes
buffered_command = byte_command + bytes([0xCD] * (64 - len(byte_command)))
assert len(buffered_command) == 64 and config

try:
    with hid.Device(path=config['path']) as device:
        # Send the buffered command as a byte array
        device.write(buffered_command)
        print("Command sent successfully.")
except IOError as e:
    print(f"Error opening HID device: {e}")

Command sent successfully.
